In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('/content/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns = ['PassengerId','Name','Ticket','Cabin'],inplace = True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


# **step 1 -> train/test/split**

In [5]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns = ['Survived']),df['Survived'],test_size = 0.2,random_state = 42)
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [7]:
X_test.head(5)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
709,3,male,NaN,1,1,15.2458,C
439,2,male,31.0,0,0,10.5000,S
840,3,male,20.0,0,0,7.9250,S
720,2,female,6.0,0,1,33.0000,S
39,3,female,14.0,1,0,11.2417,C


In [8]:
y_train.sample(5)

,Survived
186,1
483,1
154,0
371,0
38,0


# **imputation transformer**

In [9]:
trf1 = ColumnTransformer([('impute_age',SimpleImputer(),[2]),
                          ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])],
                         remainder = 'passthrough')

# **One Hot Encoding**

In [11]:
trf2 = ColumnTransformer([('ohe_sex_embarked',OneHotEncoder(sparse_output = False,handle_unknown = 'ignore'),[1,6])],
                         remainder = 'passthrough')

# **Scaling**

In [12]:
trf3 = ColumnTransformer([('scale',MinMaxScaler(),slice(0,10))])

# **Feature Selection**

In [13]:
trf4 = SelectKBest(score_func = chi2,k = 8)

# **train the model**

In [14]:
trf5 = DecisionTreeClassifier()

# **Create Pipeline**

In [15]:
pipe = Pipeline([
                  ('trf1',trf1),
                  ('trf2',trf2),
                  ('trf3',trf3),
                  ('trf4',trf4),
                  ('trf5',trf5)
])

# **Make Pipeline**

In [16]:
##Alternate Syntax
Pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

In [ ]:
#train
pipe.fit(X_train,y_train)

# **Explore the Pipeline**

In [29]:
pipe.named_steps
pipe.named_steps['trf1'].transformers_[0][1].statistics_


AttributeError: 'SimpleImputer' object has no attribute 'statistics'

# **Display Pipeline**

In [22]:
from sklearn import set_config
set_config(display='diagram')

# **Predict**

In [23]:
y_pred = pipe.predict(X_test)
y_pred

NotFittedError: Pipeline is not fitted yet.